<a href="https://colab.research.google.com/github/saurabh0512/mnist/blob/master/MNIST(pytorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install torch torchvision

In [0]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
args={}
kwargs={}
args['batch_size']=1000
args['test_batch_size']=1000
args['epochs']=30  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.01 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
args['log_interval']=10
args['cuda']=False


In [0]:
#loading training data with transformation
train_loader =datasets.MNIST('../data', train=True, download=True,transform=transforms.Compose([transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))]))
#loading test data with transformation
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))])),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)

0it [00:00, ?it/s]

9920512it [00:01, 8616822.14it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 134372.57it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2184922.76it/s]                            
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 49192.09it/s]            


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
split=int(0.8*len(train_loader))
index_list=list(range(len(train_loader)))
train_idx,valid_idx=index_list[:split],index_list[split:]

tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

train_loader1=torch.utils.data.DataLoader(train_loader,batch_size=args['batch_size'],sampler=tr_sampler,**kwargs)
val_loader=torch.utils.data.DataLoader(train_loader,batch_size=args['batch_size'],sampler=val_sampler,**kwargs)

In [0]:
class Net(nn.Module):# base class
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop=nn.Dropout2d()
    self.fc1=nn.Linear(320,50)
    self.fc2=nn.Linear(50,10)
    
  def forward(self, x):
    #Convolutional Layer with Pooling Layer and Activation
    x = F.relu(F.max_pool2d(self.conv1(x), 2)) 
    #Convolutional Layer with Dropout,Pooling Layer and Activation
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 320)
    #Fully Connected Layer and Activation
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    #Fully Connected Layer and Activation
    x = self.fc2(x)
    #Softmax gets probabilities. 
    return F.log_softmax(x, dim=1)

In [0]:
def train(epoch):
  model.train()
  for batch_idx,(data,target) in enumerate(train_loader1):
    if args['cuda']:
      data,target= data.cuda(),target.cuda()
    data,target=Variable(data),Variable(target) #variable helps storing gradiant and points to function that called it
    optimizer.zero_grad() #it zeros out all the gradiant ,as gradiant has memory
    
    output=model(data) #call to Net() class
    
    loss=F.nll_loss(output,target) #calculation using loss negative log likelihood (-log(y)) 
    
    loss.backward() #calculation of gradiant of loss with respect to all previous variables
    
    optimizer.step()
    if batch_idx % args['log_interval'] ==0: #print after every 10,000 samples 
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\ttrainingLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader1.sampler),
                100. * batch_idx / len(train_loader1.sampler), loss.item()))

In [0]:
def valid(epoch):
  model.eval()
  for batch_idx,(data,target) in enumerate(val_loader):
    if args['cuda']:
      data,target= data.cuda(),target.cuda()
    data,target=Variable(data),Variable(target)
    output=model(data)
    loss=F.nll_loss(output,target)
    loss.backward()
    optimizer.step()
    if batch_idx % args['log_interval'] ==0: #print after every 10,000 samples 
      print('Validation Epoch: {} [{}/{} ({:.0f}%)]\tvalidationLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(val_loader.sampler),
                100. * batch_idx / len(val_loader.sampler), loss.item()))

In [0]:
def test():
    model.eval()
    test_loss=0
    correct = 0
    with torch.no_grad(): 
      for data, target in test_loader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
#         for i in range(0,len(data)-1):
#           if pred[i]!=target[i]:
#             print('Incorrect prediction{} for Target{}'.format(pred[i],target[i]))
        correct += pred.eq(target.data.view_as(pred)).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
#     for param in model.parameters():
#       print(param.data)


In [0]:
model=Net()
if args['cuda']:
  model.cuda() #load whole module onto GPU
  
optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])

for epoch in range(1, args['epochs'] + 1):
  train(epoch)
  valid(epoch)
  test()


Train Epoch: 1 [0/48000 (0%)]	trainingLoss: 2.316601
Train Epoch: 1 [10000/48000 (0%)]	trainingLoss: 2.304163
Train Epoch: 1 [20000/48000 (0%)]	trainingLoss: 2.283593
Train Epoch: 1 [30000/48000 (0%)]	trainingLoss: 2.273845
Train Epoch: 1 [40000/48000 (0%)]	trainingLoss: 2.272114
Validation Epoch: 1 [0/12000 (0%)]	validationLoss: 2.235923
Validation Epoch: 1 [10000/12000 (0%)]	validationLoss: 1.901967


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Incorrect predictiontensor([2]) for Target3
Incorrect predictiontensor([6]) for Target4
Incorrect predictiontensor([6]) for Target8
Incorrect predictiontensor([4]) for Target2
Incorrect predictiontensor([2]) for Target3
Incorrect predictiontensor([4]) for Target9
Incorrect predictiontensor([2]) for Target8
Incorrect predictiontensor([6]) for Target3
Incorrect predictiontensor([4]) for Target9
Incorrect predictiontensor([4]) for Target5
Incorrect predictiontensor([4]) for Target9
Incorrect predictiontensor([8]) for Target3
Incorrect predictiontensor([8]) for Target3
Incorrect predictiontensor([4]) for Target5
Incorrect predictiontensor([0]) for Target5
Incorrect predictiontensor([8]) for Target5
Incorrect predictiontensor([8]) for Target3
Incorrect predictiontensor([4]) for Target9
Incorrect predictiontensor([8]) for Target3
Incorrect predictiontensor([6]) for Target9
Incorrect predictiontensor([4]) for Target9
Incorrect predictiontensor([8]) for Target3
Incorrect predictiontensor([6]) 